# Notebook 5: Upload Assemblies<a class="tocSkip">

**Upload the final reassembled fastas as a submission**
    
    
**The steps that we will take are:**
1. Import Statements & Global Variable Definitions
2. Upload Final Assemblies
3. Upload Raw Assemblies + GFAs
4. Upload Unitig GFAs

# Import Statements & Global Variable Definitions

In [23]:
%%capture
%pip install --upgrade --no-cache-dir git+https://github.com/DataBiosphere/ssds

In [1]:
import terra_notebook_utils as tnu
import terra_pandas as tp
import pandas as pd
import os
from ssds import deployment

## Global Variable Definitions

In [2]:
## submission_id is generated by uuid (gives unique ID to each submission)
submission_id       = "CDCCF569-B54B-4C20-A6AC-756A414B4AAB"

## submission_name lends some recognizability to the submission
submission_name     = "YEAR_1_ASSEMBLIES_V2"

ds = deployment._GSStaging(google_billing_project="firecloud-cgl")   # _GSStaging()
destintation_bucket = "gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf" # _GSStaging()

## Where the submission files will actually land (we may add one directory level later)
dest_full_path      = os.path.join(destintation_bucket, "submissions", submission_id + "--" + submission_name)

## Set Environment Variables

In [3]:
# Get the Google billing project name and workspace name
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE =os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_Reassembly
Workspace storage bucket: gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/


# Upload Final Assemblies

## Read In Data Table w/ Final Assemblies

In [4]:
final_assemblies_df = tp.table_to_dataframe("corrected_sample")

final_assemblies_df.head()

,pat_masked_cleaned_fa,corrected_MT_contig,mat_masked_cleaned_fa,mat_final_fasta,sample_name,pat_final_fasta
corrected_sample_id,,,,,,
HG002_downsampled,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,HG002,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...
HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,HG005,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...
HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/3...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/1...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/5...,HG00621,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...


## Upload Directly From GCP w/ SSDS

In [ ]:
## Loop through rows of data table, return tuple of values
for index, row in final_assemblies_df.iterrows():
    
    sample_id   = row.name
    sample_name = row['sample_name']
    
    ## We are not transferring the HG002 downsampled data (just the full set)
    if sample_id == "HG002_downsampled":
        continue
    
    else:
        mat_fa_fp = row['mat_final_fasta']
        pat_fa_fp = row['pat_final_fasta']

        mat_fa_fn = os.path.basename(mat_fa_fp)
        pat_fa_fn = os.path.basename(pat_fa_fp)
        
        mat_subm_path = f"{sample_name}/assemblies/year1_freeze_assembly_v2/{mat_fa_fn}"
        pat_subm_path = f"{sample_name}/assemblies/year1_freeze_assembly_v2/{pat_fa_fn}"
                
        ds.copy(mat_fa_fp, submission_id, submission_name, mat_subm_path)
        ds.copy(pat_fa_fp, submission_id, submission_name, pat_subm_path)

# Upload Raw Assemblies + GFAs
## Read In Sample Data Table

In [5]:
sample_df = tp.table_to_dataframe("sample")

sample_df.head()

,QCstats,mat_ilmn,hifi,paternal_id,matYak,maternal_id,patYak,nanopore,pat_ilmn,maternalFastaGz,...,adapterBlockListTarGz,countReadsTarGz,binFilesTarGz,hic,bionano_cmap,bionano_bnx,strandseq,notes,patAssemblyChm13WinnowmapBam,matAssemblyChm13WinnowmapBam
sample_id,,,,,,,,,,,,,,,,,,,,,
HG002_chem_v2,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/b...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG003,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,HG004,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/6...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HG002_downsampled,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/b...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG003,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,HG004,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/s...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HG002_full_v0.14,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/f...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG003,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,HG004,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/f...,...,NaN,NaN,NaN,[],[],[],[],NaN,NaN,NaN
HG002_full_v0.14.1,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/e...,NaN,NaN,HG003,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,HG004,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,NaN,NaN,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/H...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HG00438,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/b...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00436,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,HG00437,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/y...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[],NaN,NaN,NaN


In [6]:
## Select just the columns that we need
sample_df = sample_df[['maternalContigGfaTarGz', 'paternalContigGfaTarGz', 'maternalFastaGz', 'paternalFastaGz', 'rawUnitigGfaTarGz']]

In [7]:
## Merge to select for rows that were assembled
df_merged = sample_df.merge(final_assemblies_df, how='right', 
                            left_index=True,
                            right_on="sample_name")

In [8]:
## Pull in raw GFAs for HG02080 (not stored in split row)
index_list = df_merged.index.tolist()
idx        = index_list.index('HG02080_split')

index_list[idx] = 'HG02080'
df_merged.index = index_list

## Upload GFAs

In [30]:
! mkdir raw_assembly
%cd raw_assembly

mkdir: cannot create directory ‘raw_assembly’: File exists
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly


In [32]:
## Loop through rows of data table, return tuple of values
for index, row in df_merged.iterrows():
    
    sample_id   = row.name
    sample_name = row['sample_name']
    
    ## We are not transferring the HG002 downsampled data (just the full set)
    if sample_id == "HG002_downsampled":
        continue
    
    else:
        
        ! mkdir -p {sample_name}/hifiasm_v0.14_raw
        %cd {sample_name}/hifiasm_v0.14_raw
        
        ## Get GFAs & Unzip (We need to add GFAs for HG002 manually)
        if sample_id != "HG002_full_v0.14":
            
            mat_gfa_fp = row['maternalContigGfaTarGz']
            pat_gfa_fp = row['paternalContigGfaTarGz']

            mat_gfa_fn = os.path.basename(mat_gfa_fp)
            pat_gfa_fn = os.path.basename(pat_gfa_fp)

            ! gsutil cp {mat_gfa_fp} .
            ! gsutil cp {pat_gfa_fp} .

            ! tar -xvf {mat_gfa_fn}
            ! tar -xvf {pat_gfa_fn}
        
            ! rm {mat_gfa_fn}
            ! rm {pat_gfa_fn}
            
        ## Get fastas
        mat_fa_fp  = row['maternalFastaGz']
        pat_fa_fp  = row['paternalFastaGz']
        
        ! gsutil cp {mat_fa_fp} .
        ! gsutil cp {pat_fa_fp} .
        
        %cd ../..

/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG002/hifiasm_v0.14_raw
CommandException: No URLs matched: nan
CommandException: No URLs matched: nan
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG00438/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/05ee8da1-2202-458d-9eba-83c5df92cd33/call-trioHifiasm/runTrioHifiasm/3abc12a2-4fe2-418f-a6e1-2f2887e87c38/call-trioHifiasm/HG00438.mat.contig_gfa.tar.gz...
/ [1 files][803.7 MiB/803.7 MiB]   39.4 MiB/s                                   
Operation completed over 1 objects/803.7 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/05ee8da1-2202-458d-9eba-83c5df92cd33/call-trioHifiasm/runTrioHifiasm/3abc12a2-4fe2-418f-a6e1-2f2887e87c38/call-trioHifiasm/HG00438.pat.contig_gf

/ [1 files][813.8 MiB/813.8 MiB]   41.4 MiB/s                                   
Operation completed over 1 objects/813.8 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG00733/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/a76270be-306c-465d-8b4a-d9d53bf16441/call-trioHifiasm/runTrioHifiasm/6af88ecd-cb3b-44c9-bdd1-ace6f5a3781e/call-trioHifiasm/HG00733.mat.contig_gfa.tar.gz...
\ [1 files][828.9 MiB/828.9 MiB]   19.7 MiB/s                                   
Operation completed over 1 objects/828.9 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/a76270be-306c-465d-8b4a-d9d53bf16441/call-trioHifiasm/runTrioHifiasm/6af88ecd-cb3b-44c9-bdd1-ace6f5a3781e/call-trioHifiasm/HG00733.pat.contig_gfa.

/ [1 files][847.9 MiB/847.9 MiB]   29.3 MiB/s                                   
Operation completed over 1 objects/847.9 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG01243/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/5b1f2453-54fe-4da7-8d8c-db485b1ba17d/call-trioHifiasm/runTrioHifiasm/d37602e4-82c1-4db5-9722-cd563fc402fe/call-trioHifiasm/attempt-2/HG01243.mat.contig_gfa.tar.gz...
\ [1 files][812.9 MiB/812.9 MiB]   28.0 MiB/s                                   
Operation completed over 1 objects/812.9 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/5b1f2453-54fe-4da7-8d8c-db485b1ba17d/call-trioHifiasm/runTrioHifiasm/d37602e4-82c1-4db5-9722-cd563fc402fe/call-trioHifiasm/attempt-2/HG0

- [1 files][846.2 MiB/846.2 MiB]   41.6 MiB/s                                   
Operation completed over 1 objects/846.2 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG01891/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/a1525bbb-2bd5-4cea-b3b2-4736acf41eaf/call-trioHifiasm/runTrioHifiasm/213f750d-07ed-4d7b-819a-e89d6c046b80/call-trioHifiasm/HG01891.mat.contig_gfa.tar.gz...
/ [1 files][835.4 MiB/835.4 MiB]   37.8 MiB/s                                   
Operation completed over 1 objects/835.4 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/a1525bbb-2bd5-4cea-b3b2-4736acf41eaf/call-trioHifiasm/runTrioHifiasm/213f750d-07ed-4d7b-819a-e89d6c046b80/call-trioHifiasm/HG01891.pat.contig_gfa.

/ [1 files][852.5 MiB/852.5 MiB]   43.1 MiB/s                                   
Operation completed over 1 objects/852.5 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02055/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/9b670e97-8a74-4494-8962-6e15c05bcb63/call-trioHifiasm/runTrioHifiasm/0efb3b7d-fd20-42a4-8174-1d59291d33ab/call-trioHifiasm/attempt-2/HG02055.mat.contig_gfa.tar.gz...
\ [1 files][810.3 MiB/810.3 MiB]   40.3 MiB/s                                   
Operation completed over 1 objects/810.3 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/9b670e97-8a74-4494-8962-6e15c05bcb63/call-trioHifiasm/runTrioHifiasm/0efb3b7d-fd20-42a4-8174-1d59291d33ab/call-trioHifiasm/attempt-2/HG0

\ [1 files][819.3 MiB/819.3 MiB]   28.5 MiB/s                                   
Operation completed over 1 objects/819.3 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02148/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/05e671ff-9eaa-4d0d-be6a-5eec093663aa/call-trioHifiasm/runTrioHifiasm/76e2d327-aee8-4d12-bc1b-63ae6688ad34/call-trioHifiasm/HG02148.mat.contig_gfa.tar.gz...
\ [1 files][813.2 MiB/813.2 MiB]   29.7 MiB/s                                   
Operation completed over 1 objects/813.2 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/05e671ff-9eaa-4d0d-be6a-5eec093663aa/call-trioHifiasm/runTrioHifiasm/76e2d327-aee8-4d12-bc1b-63ae6688ad34/call-trioHifiasm/HG02148.pat.contig_gfa.

| [1 files][848.9 MiB/848.9 MiB]   40.0 MiB/s                                   
Operation completed over 1 objects/848.9 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02572/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/535e9ad7-3931-4e7e-bca4-b46069d59f71/call-trioHifiasm/runTrioHifiasm/03bff272-3f64-453b-8a65-61bf5f58fbec/call-trioHifiasm/HG02572.mat.contig_gfa.tar.gz...
| [1 files][816.8 MiB/816.8 MiB]   31.9 MiB/s                                   
Operation completed over 1 objects/816.8 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/535e9ad7-3931-4e7e-bca4-b46069d59f71/call-trioHifiasm/runTrioHifiasm/03bff272-3f64-453b-8a65-61bf5f58fbec/call-trioHifiasm/HG02572.pat.contig_gfa.

- [1 files][817.6 MiB/817.6 MiB]   41.0 MiB/s                                   
Operation completed over 1 objects/817.6 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02723/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/6fb21d79-532a-43cc-a24a-073f0be53642/call-trioHifiasm/runTrioHifiasm/1ba7d7f3-9dcf-4388-a05a-d82144c53861/call-trioHifiasm/HG02723.mat.contig_gfa.tar.gz...
\ [1 files][819.4 MiB/819.4 MiB]   36.0 MiB/s                                   
Operation completed over 1 objects/819.4 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/6fb21d79-532a-43cc-a24a-073f0be53642/call-trioHifiasm/runTrioHifiasm/1ba7d7f3-9dcf-4388-a05a-d82144c53861/call-trioHifiasm/HG02723.pat.contig_gfa.

\ [1 files][854.8 MiB/854.8 MiB]   23.0 MiB/s                                   
Operation completed over 1 objects/854.8 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03540/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/59fa1498-4eed-441a-9fe9-5f264db4ea58/call-trioHifiasm/runTrioHifiasm/487d5b7f-b598-4eae-9825-1d6fb32381cd/call-trioHifiasm/attempt-2/HG03540.mat.contig_gfa.tar.gz...
/ [1 files][814.5 MiB/814.5 MiB]   28.0 MiB/s                                   
Operation completed over 1 objects/814.5 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/59fa1498-4eed-441a-9fe9-5f264db4ea58/call-trioHifiasm/runTrioHifiasm/487d5b7f-b598-4eae-9825-1d6fb32381cd/call-trioHifiasm/attempt-2/HG0

| [1 files][852.2 MiB/852.2 MiB]   36.6 MiB/s                                   
Operation completed over 1 objects/852.2 MiB.                                    
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA20129/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/1fa787e9-5f82-4d41-8d39-72359af44b99/call-trioHifiasm/runTrioHifiasm/5bc3510f-bbc8-476c-a79f-9305e77c2104/call-trioHifiasm/NA20129.mat.contig_gfa.tar.gz...
/ [1 files][813.2 MiB/813.2 MiB]   39.9 MiB/s                                   
Operation completed over 1 objects/813.2 MiB.                                    
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/1fa787e9-5f82-4d41-8d39-72359af44b99/call-trioHifiasm/runTrioHifiasm/5bc3510f-bbc8-476c-a79f-9305e77c2104/call-trioHifiasm/NA20129.pat.contig_gfa.

## Manually pull in GFAs for HG002 full data set (Done external to AnVIL)

In [41]:
%cd /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly

/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly


In [42]:
%cd HG002/hifiasm_v0.14_raw

!mkdir HG002.pat.contig_gfa
%cd HG002.pat.contig_gfa

! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap1.p_ctg.gfa .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap1.p_ctg.lowQ.bed .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap1.p_ctg.noseq.gfa .

%cd ..
!mkdir HG002.mat.contig_gfa
%cd HG002.mat.contig_gfa

! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap2.p_ctg.gfa .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap2.p_ctg.lowQ.bed .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap2.p_ctg.noseq.gfa .

/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG002/hifiasm_v0.14_raw
mkdir: cannot create directory ‘HG002.pat.contig_gfa’: File exists
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG002/hifiasm_v0.14_raw/HG002.pat.contig_gfa
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap1.p_ctg.gfa...
| [1 files][  2.8 GiB/  2.8 GiB]   46.8 MiB/s                                   
Operation completed over 1 objects/2.8 GiB.                                      
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.hap1.p_ctg.lowQ.bed...
/ [1 files][  8.7 MiB/  8.7 MiB]                                                
Operation completed over 1 objects/8.7 MiB.                                      
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--

##  Upload GFAs As A Submission

In [44]:
%cd /home/jupyter-user/notebooks/HPRC_Reassembly/edit/

/home/jupyter-user/notebooks/HPRC_Reassembly/edit


In [48]:
! ssds staging upload \
    --deployment gcp \
    --submission-id CDCCF569-B54B-4C20-A6AC-756A414B4AAB \
    raw_assembly

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA18906/hifiasm_v0.14_raw/NA18906.pat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_raw/NA18906.pat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA18906/hifiasm_v0.14_raw/NA18906.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_raw/NA18906.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA18906/hifiasm_v0.14_raw/NA18906.pat.contig_gfa/NA18906.hap1.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_raw/NA18906.pat.contig_gfa/NA18906.hap1.p_ctg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/not

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02886/hifiasm_v0.14_raw/HG02886.pat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02886/hifiasm_v0.14_raw/HG02886.pat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02886/hifiasm_v0.14_raw/HG02886.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02886/hifiasm_v0.14_raw/HG02886.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02886/hifiasm_v0.14_raw/HG02886.mat.contig_gfa/HG02886.hap2.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02886/hifiasm_v0.14_raw/HG02886.mat.contig_gfa/HG02886.hap2.p_ctg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/noteb

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA20129/hifiasm_v0.14_raw/NA20129.pat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA20129/hifiasm_v0.14_raw/NA20129.pat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA20129/hifiasm_v0.14_raw/NA20129.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA20129/hifiasm_v0.14_raw/NA20129.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/NA20129/hifiasm_v0.14_raw/NA20129.pat.contig_gfa/NA20129.hap1.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA20129/hifiasm_v0.14_raw/NA20129.pat.contig_gfa/NA20129.hap1.p_ctg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/not

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG005/hifiasm_v0.14_raw/HG005.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG005/hifiasm_v0.14_raw/HG005.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG005/hifiasm_v0.14_raw/HG005.pat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG005/hifiasm_v0.14_raw/HG005.pat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG005/hifiasm_v0.14_raw/HG005.mat.contig_gfa/HG005.hap2.p_ctg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG005/hifiasm_v0.14_raw/HG005.mat.contig_gfa/HG005.hap2.p_ctg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03492/hifiasm_v0.14_raw/HG03492.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03492/hifiasm_v0.14_raw/HG03492.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03492/hifiasm_v0.14_raw/HG03492.pat.contig_gfa/HG03492.hap1.p_ctg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03492/hifiasm_v0.14_raw/HG03492.pat.contig_gfa/HG03492.hap1.p_ctg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03492/hifiasm_v0.14_raw/HG03492.pat.contig_gfa/HG03492.hap1.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03492/hifiasm_v0.14_raw/HG03492.pat.contig_gfa/HG03492.hap1.p_ctg.n

...

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG01258/hifiasm_v0.14_raw/HG01258.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01258/hifiasm_v0.14_raw/HG01258.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG01258/hifiasm_v0.14_raw/HG01258.mat.contig_gfa/HG01258.hap2.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01258/hifiasm_v0.14_raw/HG01258.mat.contig_gfa/HG01258.hap2.p_ctg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG01258/hifiasm_v0.14_raw/HG01258.mat.contig_gfa/HG01258.hap2.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01258/hifiasm_v0.14_raw/HG01258.mat.contig_gfa/HG01258.h

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03453/hifiasm_v0.14_raw/HG03453.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03453/hifiasm_v0.14_raw/HG03453.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03453/hifiasm_v0.14_raw/HG03453.mat.contig_gfa/HG03453.hap2.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03453/hifiasm_v0.14_raw/HG03453.mat.contig_gfa/HG03453.hap2.p_ctg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03453/hifiasm_v0.14_raw/HG03453.mat.contig_gfa/HG03453.hap2.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03453/hifiasm_v0.14_raw/HG03453.mat.contig_gfa/HG03453.h

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02080/hifiasm_v0.14_raw/HG02080.pat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02080/hifiasm_v0.14_raw/HG02080.pat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02080/hifiasm_v0.14_raw/HG02080.pat.contig_gfa/HG02080.hap1.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02080/hifiasm_v0.14_raw/HG02080.pat.contig_gfa/HG02080.hap1.p_ctg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02080/hifiasm_v0.14_raw/HG02080.pat.contig_gfa/HG02080.hap1.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02080/hifiasm_v0.14_raw/HG02080.pat.contig_gfa/HG02080.h

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02109/hifiasm_v0.14_raw/HG02109.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02109/hifiasm_v0.14_raw/HG02109.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02109/hifiasm_v0.14_raw/HG02109.mat.contig_gfa/HG02109.hap2.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02109/hifiasm_v0.14_raw/HG02109.mat.contig_gfa/HG02109.hap2.p_ctg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG02109/hifiasm_v0.14_raw/HG02109.mat.contig_gfa/HG02109.hap2.p_ctg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG02109/hifiasm_v0.14_raw/HG02109.mat.contig_gfa/HG02109.ha

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03516/hifiasm_v0.14_raw/HG03516.mat.fa.gz to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03516/hifiasm_v0.14_raw/HG03516.mat.fa.gz
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03516/hifiasm_v0.14_raw/HG03516.mat.contig_gfa/HG03516.hap2.p_ctg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03516/hifiasm_v0.14_raw/HG03516.mat.contig_gfa/HG03516.hap2.p_ctg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/raw_assembly/HG03516/hifiasm_v0.14_raw/HG03516.mat.contig_gfa/HG03516.hap2.p_ctg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03516/hifiasm_v0.14_raw/HG03516.mat.contig_gfa/HG03516.hap2.p_ctg.lo

# Upload Unitig GFAs

## Download Unitig GFAs

In [10]:
%cd /home/jupyter-user/notebooks/HPRC_Reassembly/edit

/home/jupyter-user/notebooks/HPRC_Reassembly/edit


In [11]:
! mkdir unitigs
%cd unitigs

/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs


In [13]:
## Loop through rows of data table, return tuple of values
for index, row in df_merged.iterrows():
    
    sample_id   = row.name
    sample_name = row['sample_name']
    
    ## We are not transferring the HG002 downsampled data (just the full set)
    if sample_id == "HG002_downsampled":
        continue
    
    else:
        
        ! mkdir -p {sample_name}/hifiasm_v0.14_raw
        %cd {sample_name}/hifiasm_v0.14_raw
        
        ## Get GFAs & Unzip (We need to add GFAs for HG002 manually)
        if sample_id != "HG002_full_v0.14":
            
            utg_gfa_fp = row['rawUnitigGfaTarGz']
            utg_gfa_fn = os.path.basename(utg_gfa_fp)

            ! gsutil cp {utg_gfa_fp} .

            ! tar -xvf {utg_gfa_fn}
            ! rm {utg_gfa_fn}
        
        %cd ../..

/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG00438/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/05ee8da1-2202-458d-9eba-83c5df92cd33/call-trioHifiasm/runTrioHifiasm/3abc12a2-4fe2-418f-a6e1-2f2887e87c38/call-trioHifiasm/HG00438.raw_unitig_gfa.tar.gz...
| [1 files][  2.0 GiB/  2.0 GiB]   48.8 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      
HG00438.raw_unitig_gfa/
HG00438.raw_unitig_gfa/HG00438.dip.r_utg.gfa
HG00438.raw_unitig_gfa/HG00438.dip.r_utg.noseq.gfa
HG00438.raw_unitig_gfa/HG00438.dip.r_utg.lowQ.bed
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG005/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b9

HG01123.raw_unitig_gfa/HG01123.dip.r_utg.lowQ.bed
HG01123.raw_unitig_gfa/HG01123.dip.r_utg.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01175/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/91800035-6097-402e-84e1-393ded7d2ffe/call-trioHifiasm/runTrioHifiasm/5e892a17-52f1-4811-bbcc-1207cb246d28/call-trioHifiasm/HG01175.raw_unitig_gfa.tar.gz...
- [1 files][  2.0 GiB/  2.0 GiB]   45.6 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      
HG01175.raw_unitig_gfa/
HG01175.raw_unitig_gfa/HG01175.dip.r_utg.gfa
HG01175.raw_unitig_gfa/HG01175.dip.r_utg.noseq.gfa
HG01175.raw_unitig_gfa/HG01175.dip.r_utg.lowQ.bed
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01243/hifiasm_v0.14_raw
Copying gs://fc-0c2122

HG02080.raw_unitig_gfa/HG02080.dip.r_utg.noseq.gfa
HG02080.raw_unitig_gfa/HG02080.dip.r_utg.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG02109/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/e989e67e-14bd-487a-ad73-2f1b7a24bf4d/call-trioHifiasm/runTrioHifiasm/c099567e-1b00-42c4-898c-eef876cd7da6/call-trioHifiasm/HG02109.raw_unitig_gfa.tar.gz...
| [1 files][  1.9 GiB/  1.9 GiB]   42.3 MiB/s                                   
Operation completed over 1 objects/1.9 GiB.                                      
HG02109.raw_unitig_gfa/
HG02109.raw_unitig_gfa/HG02109.dip.r_utg.noseq.gfa
HG02109.raw_unitig_gfa/HG02109.dip.r_utg.lowQ.bed
HG02109.raw_unitig_gfa/HG02109.dip.r_utg.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG02145/hifiasm_v0.14_raw
Copying gs://fc-0c212

HG02723.raw_unitig_gfa/HG02723.dip.r_utg.lowQ.bed
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG02818/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/4ce1201c-412d-4b06-9b98-e19b9850b98f/call-trioHifiasm/runTrioHifiasm/97d3d9fc-dd1a-470c-a042-74592008504d/call-trioHifiasm/HG02818.raw_unitig_gfa.tar.gz...
- [1 files][  2.0 GiB/  2.0 GiB]   34.6 MiB/s                                   
Operation completed over 1 objects/2.0 GiB.                                      
HG02818.raw_unitig_gfa/
HG02818.raw_unitig_gfa/HG02818.dip.r_utg.lowQ.bed
HG02818.raw_unitig_gfa/HG02818.dip.r_utg.gfa
HG02818.raw_unitig_gfa/HG02818.dip.r_utg.noseq.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG02886/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-

NA19240.raw_unitig_gfa/NA19240.dip.r_utg.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/NA20129/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-854f-75e896c5c28a/trioHifiasmAssembly/1fa787e9-5f82-4d41-8d39-72359af44b99/call-trioHifiasm/runTrioHifiasm/5bc3510f-bbc8-476c-a79f-9305e77c2104/call-trioHifiasm/NA20129.raw_unitig_gfa.tar.gz...
/ [1 files][  1.9 GiB/  1.9 GiB]   45.3 MiB/s                                   
Operation completed over 1 objects/1.9 GiB.                                      
NA20129.raw_unitig_gfa/
NA20129.raw_unitig_gfa/NA20129.dip.r_utg.lowQ.bed
NA20129.raw_unitig_gfa/NA20129.dip.r_utg.gfa
NA20129.raw_unitig_gfa/NA20129.dip.r_utg.noseq.gfa
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/NA21309/hifiasm_v0.14_raw
Copying gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/c478c8fb-f903-4a0e-

## Manually pull in GFAs for HG002 full data set (Done external to AnVIL)

In [20]:
%cd HG002/hifiasm_v0.14_raw

!mkdir HG002.raw_unitig_gfa
%cd HG002.raw_unitig_gfa

! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.gfa .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.lowQ.bed .
! gsutil -u firecloud-cgl cp gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.noseq.gfa .


/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw
/home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.gfa...
- [1 files][  7.1 GiB/  7.1 GiB]   28.0 MiB/s                                   
Operation completed over 1 objects/7.1 GiB.                                      
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.lowQ.bed...
- [1 files][ 45.0 MiB/ 45.0 MiB]                                                
Operation completed over 1 objects/45.0 MiB.                                     
Copying gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/4DE3A234-8903-44C7-8BEF-86C5AF73F5B1--HG002-ASSEMBLY/HG002-full-0.14.dip.r_utg.noseq.gfa...
- [1 files][170.9 MiB/170

##  Upload GFAs As A Submission

In [21]:
%cd /home/jupyter-user/notebooks/HPRC_Reassembly/edit/

/home/jupyter-user/notebooks/HPRC_Reassembly/edit


In [24]:
! ssds staging upload \
    --deployment gcp \
    --submission-id CDCCF569-B54B-4C20-A6AC-756A414B4AAB \
    unitigs

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/NA18906/hifiasm_v0.14_raw/NA18906.raw_unitig_gfa/NA18906.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_raw/NA18906.raw_unitig_gfa/NA18906.dip.r_utg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/NA18906/hifiasm_v0.14_raw/NA18906.raw_unitig_gfa/NA18906.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_raw/NA18906.raw_unitig_gfa/NA18906.dip.r_utg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/NA18906/hifiasm_v0.14_raw/NA18906.raw_unitig_gfa/NA18906.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/NA18906/hifiasm_v0.14_r

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01071/hifiasm_v0.14_raw/HG01071.raw_unitig_gfa/HG01071.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01071/hifiasm_v0.14_raw/HG01071.raw_unitig_gfa/HG01071.dip.r_utg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01071/hifiasm_v0.14_raw/HG01071.raw_unitig_gfa/HG01071.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01071/hifiasm_v0.14_raw/HG01071.raw_unitig_gfa/HG01071.dip.r_utg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01071/hifiasm_v0.14_raw/HG01071.raw_unitig_gfa/HG01071.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01071/hifiasm_v

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03579/hifiasm_v0.14_raw/HG03579.raw_unitig_gfa/HG03579.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03579/hifiasm_v0.14_raw/HG03579.raw_unitig_gfa/HG03579.dip.r_utg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03579/hifiasm_v0.14_raw/HG03579.raw_unitig_gfa/HG03579.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03579/hifiasm_v0.14_raw/HG03579.raw_unitig_gfa/HG03579.dip.r_utg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03579/hifiasm_v0.14_raw/HG03579.raw_unitig_gfa/HG03579.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03579/hifiasm_v0.14_r

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa/HG002-full-0.14.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa/HG002-full-0.14.dip.r_utg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa/HG002-full-0.14.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa/HG002-full-0.14.dip.r_utg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG002/hifiasm_v0.14_raw/HG002.raw_unitig_gfa/HG002-full-0.14.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03098/hifiasm_v0.14_raw/HG03098.raw_unitig_gfa/HG03098.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03098/hifiasm_v0.14_raw/HG03098.raw_unitig_gfa/HG03098.dip.r_utg.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03098/hifiasm_v0.14_raw/HG03098.raw_unitig_gfa/HG03098.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03098/hifiasm_v0.14_raw/HG03098.raw_unitig_gfa/HG03098.dip.r_utg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG03098/hifiasm_v0.14_raw/HG03098.raw_unitig_gfa/HG03098.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG03098/hifiasm_v0.14_

INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01106/hifiasm_v0.14_raw/HG01106.raw_unitig_gfa/HG01106.dip.r_utg.noseq.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01106/hifiasm_v0.14_raw/HG01106.raw_unitig_gfa/HG01106.dip.r_utg.noseq.gfa
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01106/hifiasm_v0.14_raw/HG01106.raw_unitig_gfa/HG01106.dip.r_utg.lowQ.bed to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01106/hifiasm_v0.14_raw/HG01106.raw_unitig_gfa/HG01106.dip.r_utg.lowQ.bed
INFO:ssds.storage:Copied /home/jupyter-user/notebooks/HPRC_Reassembly/edit/unitigs/HG01106/hifiasm_v0.14_raw/HG01106.raw_unitig_gfa/HG01106.dip.r_utg.gfa to gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/submissions/CDCCF569-B54B-4C20-A6AC-756A414B4AAB--YEAR_1_ASSEMBLIES_V2/HG01106/hifiasm_v